<a href="https://colab.research.google.com/github/PristRawal/Deep_Learning/blob/main/Unet_Car.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from albumentations import Compose, Resize, Normalize
from albumentations.pytorch import ToTensorV2
from PIL import Image
import os

In [ ]:
class CarvanaDataset(torch.utils.data.Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.images = os.listdir(image_dir)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        img_path = os.path.join(self.image_dir, self.images[index])
        mask_path = os.path.join(self.mask_dir, self.images[index].replace(".jpg", "_mask.gif"))
        image = np.array(Image.open(img_path).convert("RGB"))
        mask = np.array(Image.open(mask_path).convert("L"), dtype=np.float32)
        mask[mask == 255.0] = 1.0

        if self.transform is not None:
            transformed = self.transform(image=image, mask=mask)
            image = transformed["image"]
            mask = transformed["mask"]

        return image, mask


In [ ]:
from albumentations import Compose, Resize, Normalize
from albumentations.pytorch import ToTensorV2

transform = Compose([
    Resize(256, 256),
    Normalize(mean=(0.0, 0.0, 0.0), std=(1.0, 1.0, 1.0)),
    ToTensorV2(),
])


In [ ]:
train_dataset = CarvanaDataset(
    image_dir='/content/drive/MyDrive/Car/Car_Train_Final',
    mask_dir='/content/drive/MyDrive/Car/Car_trainmask_Final',
    transform=transform
)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)


In [ ]:
class DoubleConv(nn.Module):
    def __init__(self,in_channels,out_channels):
      super(DoubleConv, self).__init__()
      self.double_conv = nn.Sequential (
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
      return self.double_conv(x)
class Down(nn.Module):
    """Downscaling with maxpool then double conv"""
    def __init__(self, in_channels, out_channels):
        super(Down, self).__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, out_channels)
        )

    def forward(self, x):
        return self.maxpool_conv(x)
class Up(nn.Module):
    """Upscaling then double conv"""
    def __init__(self, in_channels, out_channels, bilinear=True):
        super(Up, self).__init__()

        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
            self.conv = DoubleConv(in_channels, out_channels)
        else:
            self.up = nn.ConvTranspose2d(in_channels // 2, in_channels // 2, kernel_size=2, stride=2)
            self.conv = DoubleConv(in_channels, out_channels)


    def forward(self, x1, x2):
        x1 = self.up(x1)

        # crop x2 to x1 size (U-Net uses 'copy and crop')
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]
        x2 = x2[:, :, diffY // 2:x2.size()[2] - diffY // 2,
                     diffX // 2:x2.size()[3] - diffX // 2]

        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)

class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        return self.conv(x)
class UNet(nn.Module):
    def __init__(self, n_channels=1, n_classes=2, bilinear=True):
        super(UNet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.bilinear = bilinear

        self.inc = DoubleConv(n_channels, 64)
        self.down1 = Down(64, 128)
        self.down2 = Down(128, 256)
        self.down3 = Down(256, 512)
        self.down4 = Down(512, 1024)

        # Correct the bottleneck input channels
        self.bottleneck = DoubleConv(1024, 1024) if bilinear else DoubleConv(512, 1024)


        self.up1 = Up(1024 + 512, 512, bilinear)
        self.up2 = Up(512 + 256, 256, bilinear)
        self.up3 = Up(256 + 128, 128, bilinear)
        self.up4 = Up(128 + 64, 64, bilinear)

        self.outc = OutConv(64, n_classes)

    def forward(self, x):
        x1 = self.inc(x)       # 64
        x2 = self.down1(x1)    # 128
        x3 = self.down2(x2)    # 256
        x4 = self.down3(x3)    # 512
        x5 = self.down4(x4)    # 1024

        x5 = self.bottleneck(x5)

        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        logits = self.outc(x)
        return logits

In [ ]:
model = UNet(n_channels=3, n_classes=1).to('cuda')
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)


In [ ]:
def dice_score(preds, targets, threshold=0.5):
    preds = torch.sigmoid(preds)
    preds = (preds > threshold).float()
    intersection = (preds * targets).sum(dim=(1, 2, 3))
    union = preds.sum(dim=(1, 2, 3)) + targets.sum(dim=(1, 2, 3))
    dice = (2. * intersection + 1e-8) / (union + 1e-8)
    return dice.mean().item()

def iou_score(preds, targets, threshold=0.5):
    preds = torch.sigmoid(preds)
    preds = (preds > threshold).float()
    intersection = (preds * targets).sum(dim=(1, 2, 3))
    union = ((preds + targets) >= 1).float().sum(dim=(1, 2, 3))
    iou = (intersection + 1e-8) / (union + 1e-8)
    return iou.mean().item()


In [ ]:
def train_fn(loader, model, optimizer, loss_fn, device):
    model.train()
    loop = tqdm(loader, desc="Training", leave=False)

    total_loss = 0.0
    total_dice = 0.0
    total_iou = 0.0

    for data, targets in loop:
        data = data.to(device)
        targets = targets.to(device).unsqueeze(1)  # [B, 1, H, W]

        preds = model(data)
        loss = loss_fn(preds, targets)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_dice += dice_score(preds, targets)
        total_iou += iou_score(preds, targets)

        loop.set_postfix(loss=loss.item())

    n = len(loader)
    return total_loss / n, total_dice / n, total_iou / n


In [ ]:
def eval_fn(loader, model, loss_fn, device):
    model.eval()
    total_loss = 0.0
    total_dice = 0.0
    total_iou = 0.0

    with torch.no_grad():
        loop = tqdm(loader, desc="Validating", leave=False)
        for data, targets in loop:
            data = data.to(device)
            targets = targets.to(device).unsqueeze(1)

            preds = model(data)
            loss = loss_fn(preds, targets)

            total_loss += loss.item()
            total_dice += dice_score(preds, targets)
            total_iou += iou_score(preds, targets)

            loop.set_postfix(val_loss=loss.item())

    n = len(loader)
    return total_loss / n, total_dice / n, total_iou / n


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = UNet(n_channels=3, n_classes=1).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Datasets
train_dataset = CarvanaDataset(image_dir='/content/drive/MyDrive/Car/Car_Train_Final', mask_dir='/content/drive/MyDrive/Car/Car_trainmask_Final', transform=transform)
val_dataset = CarvanaDataset(image_dir='/content/drive/MyDrive/Car/Validation_img', mask_dir='/content/drive/MyDrive/Car/Validation_mask', transform=transform)

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False)

# Epochs
epochs = 5
for epoch in range(epochs):
    print(f"\nEpoch [{epoch+1}/{epochs}]")

    train_loss, train_dice, train_iou = train_fn(train_loader, model, optimizer, criterion, device)
    val_loss, val_dice, val_iou = eval_fn(val_loader, model, criterion, device)

    print(f"Train Loss: {train_loss:.4f} | Dice: {train_dice:.4f} | IoU: {train_iou:.4f}")
    print(f"Val   Loss: {val_loss:.4f} | Dice: {val_dice:.4f} | IoU: {val_iou:.4f}")



Epoch [1/5]


NameError: name 'eval_fn' is not defined

#Plottings

In [ ]:
torch.save(model.state_dict(), '/content/drive/MyDrive/unet_model.pth')